# 1 - Introduction

Nama: Kamelia Ramandha.

Batch: CODA-RMT-010  

Live Coding 1 Phase 0 - Menganalisa Data E-Commerce




# 2 - Import Library

In [3]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id_akun = 'calm-inkwell-469714-j8'
client = bigquery.Client(project=project_id_akun)

Authenticated


# 3 - Soal

## Soal 1: Berikan tabel jumlah transaksi tiap harinya (date berdasarkan created_at) selama the Look beroperasi (urutkan dari tanggal terkecil).

In [7]:
df_1 = client.query('''

    SELECT DATE(created_at) as date,
          COUNT(order_id) AS jumlah_transaksi
    FROM `bigquery-public-data.thelook_ecommerce.orders`
    GROUP BY date
    ORDER BY date ASC;


''').to_dataframe()

df_1

,date,jumlah_transaksi
0,2019-01-08,1
1,2019-01-16,1
2,2019-01-19,1
3,2019-01-20,1
4,2019-01-22,1
...,...,...
2375,2025-08-21,641
2376,2025-08-22,868
2377,2025-08-23,1479
2378,2025-08-24,628


## Soal 2: Urutkan status pengiriman dari yang paling sering terjadi hingga jarang terjadi selama the Look beroperasi selama tahun 2023.

In [28]:
df_2 = client.query(
    '''
SELECT status,
      COUNT(order_id) as jumlah_order
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE EXTRACT (YEAR from created_at) = 2023
GROUP BY status
ORDER BY jumlah_order DESC;

    '''
).to_dataframe()
df_2

,status,jumlah_order
0,Shipped,6397
1,Complete,5233
2,Processing,4139
3,Cancelled,3220
4,Returned,2085


## Soal 3: Dari negara mana saja kah pembeli di platform The Look selama semester pertama 2024?

In [69]:
df_3 = client.query(
    '''
SELECT DISTINCT(country)
FROM `bigquery-public-data.thelook_ecommerce.users` AS users
JOIN `bigquery-public-data.thelook_ecommerce.orders` AS orders ON users.id = orders.user_id
WHERE DATE(orders.created_at) BETWEEN '2024-01-01' AND '2024-06-30';

    '''
).to_dataframe()
df_3

,country
0,Japan
1,United States
2,Brasil
3,Colombia
4,Spain
5,China
6,Australia
7,France
8,Germany
9,Belgium


## Soal 4: Brand apa yang paling banyak dibeli oleh wanita? (berikan Top 1 nya saja) selama bulan Desember 2023?

In [38]:
df_4 = client.query(
    '''
SELECT products.brand,
        COUNT(orders.order_id) AS jumlah_order
FROM `bigquery-public-data.thelook_ecommerce.orders` AS orders
JOIN `bigquery-public-data.thelook_ecommerce.users` AS users
    ON orders.user_id = users.id
JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS order_items
    ON orders.order_id = order_items.order_id
JOIN `bigquery-public-data.thelook_ecommerce.products` AS products
    ON order_items.product_id = products.id
WHERE users.gender = 'F' AND
        DATE(orders.created_at) BETWEEN '2023-12-01' AND '2023-12-31'
GROUP BY products.brand
ORDER BY jumlah_order DESC
LIMIT 1;

    '''
).to_dataframe()
df_4

,brand,jumlah_order
0,Allegra K,80


## Soal 5: Pada bulan apa terjadi peak season selama tahun 2021-2023? (Hint: berdasarkan rata-rata jumlah transaksi dalam satu bulan)

In [47]:
df_5 = client.query(
    '''
SELECT month,
      AVG(total_order) AS avg_order
FROM (
    SELECT EXTRACT (MONTH FROM created_at) AS month,
          EXTRACT (YEAR FROM created_at) as year,
          COUNT(order_id) as total_order
    FROM `bigquery-public-data.thelook_ecommerce.orders` AS orders
    WHERE DATE(created_at) BETWEEN '2021-01-01' AND '2023-12-31'
    GROUP BY year, month
    )
GROUP BY month
ORDER BY avg_order DESC
LIMIT 1;

    '''
).to_dataframe()
df_5

,month,avg_order
0,12,1534.333333


## Soal 6: Siapa customer yang paling loyal di Q2 2024? (customer loyal dihitung dari jumlah transaksi paling banyak dan Q2 terhitung dari bulan April-Juni)

In [70]:
df_6 = client.query(
    '''
SELECT users.first_name,
      COUNT(order_id) AS jumlah_order
FROM `bigquery-public-data.thelook_ecommerce.orders` AS orders
JOIN `bigquery-public-data.thelook_ecommerce.users` AS users
    ON orders.user_id = users.id
WHERE DATE(orders.created_at) BETWEEN '2024-04-01' AND '2024-6-30'
GROUP BY users.first_name
ORDER BY jumlah_order DESC
LIMIT 1;

    '''
).to_dataframe()
df_6

,first_name,jumlah_order
0,Michael,165


## Soal 7: Berapa rata-rata durasi merchant handle barang (selisih hari dari created_at hingga shipped_at) dan pengiriman barang (selisih hari dari shipped_at ke delivered_at)? Periode waktu di semester pertama 2024.

In [59]:
df_7 = client.query(
    '''
SELECT  AVG(DATE_DIFF(DATE(shipped_at), DATE(created_at), DAY)) AS avg_durasi_merchant,
      AVG(DATE_DIFF(DATE(delivered_at), DATE(shipped_at), DAY)) AS avg_pengiriman_barang
FROM `bigquery-public-data.thelook_ecommerce.orders` AS orders
WHERE DATE(orders.created_at) BETWEEN '2024-01-01' AND '2024-6-30';

    '''
).to_dataframe()

df_7

,avg_durasi_merchant,avg_pengiriman_barang
0,1.398897,2.490562


## Soal 8 = Berapa total komisi yang diperoleh the Look Commerce selama Q1 2024?
komisi merupakan 10% dari profit merchant setiap transaksinya (profit yang didapatkan merchant dihitung dari selisih retail_price dengan cost dari transaksi berstatus "Complete")

In [62]:
df_8 = client.query(
    '''
SELECT SUM(products.retail_price - products.cost) * 0.10 AS komisi
FROM `bigquery-public-data.thelook_ecommerce.orders` AS orders
JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS order_items
    ON orders.order_id = order_items.order_id
JOIN `bigquery-public-data.thelook_ecommerce.products` AS products
    ON order_items.product_id = products.id
WHERE orders.status = 'Complete'
    AND DATE(orders.created_at) BETWEEN '2024-01-01' AND '2024-3-30';

    '''
).to_dataframe()

df_8

,komisi
0,8001.542509
